# Example for system in Meyers, Craig and Odde 2006

Geometry is divided into 2 domains - one surface and one volume:
- plasma membrane (PM) - cell surface
- cytosol - intracellular volume

This model has a single species, A, which is phosphorylated at the cell membrane. The unphosphorylated form of A ($A_{dephos}$) can be computed from mass conservation; everywhere $c_{A_{phos}} + c_{A_{dephos}} = c_{Tot}$, which is a constant in both time and space if the phosphorylated vs. unphosphorylated forms have the same diffusion coefficient.

There are two reactions - one in the PM and other in the cytosol. At the membrane, $A_{dephos}$ is phosphorylated by a first-order reaction with rate $k_{kin}$, and in the cytosolic volume, $A_{phos}$ is dephosphorylated by a first order reaction with rate $k_p$.

Currently, this code includes a dummy variable, kinMem, which represents relative kinase activity at the membrane, currently set as constant at 1 (dimensionless). Without this additional species, the code currently does not solve the system (it seems that there needs to be variables in at least two compartments to solve the monolithic system).


In [1]:
import os

import dolfin as d
import sympy as sym
import numpy as np

from smart import config, common, mesh, model
from smart.units import unit
from smart.model_assembly import Compartment, Parameter, Reaction, Species, sbmodel_from_locals
from matplotlib import pyplot as plt

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'


First, we define the various units for the inputs

In [2]:
# Aliases - base units
uM = unit.uM
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
# Aliases - units used in model
D_unit = um**2 / sec
flux_unit = molecule / (um**2 * sec)
vol_unit = uM
surf_unit = molecule / um**2

Next we generate the model.

In [3]:
def make_model(a,b):
    # =============================================================================================
    # Species
    # =============================================================================================
    # name, initial concentration, concentration units, diffusion, diffusion units, compartment
    Ainit = f"10*exp(-((x-{a}/4.0)**2+(y-{b}/2.0)**2)/.1)"
    Binit = f"exp(-((x-3.0*{a}/4.0)**2+(y-{b}/2.0)**2)/.1)"
    A = Species("A", Ainit, surf_unit, 1.0, D_unit, "PM1")
#     B = Species("B", Binit, surf_unit, 1.0, D_unit, "PM2")
    # kinMem = Species("kinMem", 1.0, dimensionless, 0.0, D_unit, "PM") # dummy variable; without this, the current version of SMART throws an error

    # =============================================================================================
    # Compartments
    # =============================================================================================
    # name, topological dimensionality, length scale units, marker value
    PM1 = Compartment("PM1", 2, um, 1)
#     PM2 = Compartment("PM2", 2, um, 2)
    junction = Compartment("junction", 1, um, 3)

    # =============================================================================================
    # Parameters and Reactions
    # =============================================================================================
    # Dephosphorylation of Aphos in the cytosol
    kdeg = Parameter("kdeg", 0, 1/sec)
    r1 = Reaction("r1", ["A"], [], param_map={"kon": "kdeg"},
         eqn_f_str="kon*A", species_map={"A": "A"})
#     r2 = Reaction("r2", ["B"], [], param_map={"kon": "kdeg"},
#          eqn_f_str="kon*B", species_map={"B": "B"})
    k_cross = Parameter("k_cross", 10, um/sec)
    cref = Parameter("cref", 1, surf_unit)
    r3 = Reaction("r3", ["A"], [], param_map={"kon": "k_cross", "cref": "cref"},
          eqn_f_str="kon*(A - cref)", explicit_restriction_to_domain="junction")

    # =============================================================================================
    # Gather all parameters, species, compartments and reactions
    # =============================================================================================
    return sbmodel_from_locals(locals().values())

We load the model generated above, and load in the mesh we will use in this example, iterating over 10 different values of cell radius, log-spaced between 1 and 10.

In [4]:
b = 1.; a = 5.
pc, sc, cc, rc = make_model(a,b)
# Create mesh
m = 10; n = int(a/b)*m
rect_mesh = d.RectangleMesh(d.Point(0.0, 0.0), d.Point(a, b), n, m)
mf2 = d.MeshFunction("size_t", rect_mesh, 2, 1)
for f in d.faces(rect_mesh):
    x_vals = np.zeros(3)
    idx = 0
    for vertex in d.vertices(f):
        x_vals[idx] = vertex.point().array()[0]
        idx = idx+1
    if np.mean(x_vals) > a/2.0:
        mf2[f] = 2
mf1 = d.MeshFunction("size_t", rect_mesh, 1, 0)
for e in d.edges(rect_mesh):
    x_vals = np.zeros(2)
    idx = 0
    for vertex in d.vertices(e):
        x_vals[idx] = vertex.point().array()[0]
        idx = idx+1
    if np.isclose(np.mean(x_vals), a/2.0):
        mf1[e] = 3
# Write mesh and meshfunctions to file
os.makedirs(f"rect_mesh", exist_ok=True)
filename = f"rect_mesh/rect_mesh"
hdf5 = d.HDF5File(rect_mesh.mpi_comm(), filename + ".h5", "w")
hdf5.write(rect_mesh, "/mesh")
hdf5.write(mf2, "/mf2")
hdf5.write(mf1, "/mf1")
d.File(f"{filename}_mf2.pvd") << mf2
d.File(f"{filename}_mf1.pvd") << mf1

# # Define solvers
parent_mesh = mesh.ParentMesh(
    mesh_filename=f"{filename}.h5",
    mesh_filetype="hdf5",
    name="parent_mesh",
)
configCur = config.Config()
modelCur = model.Model(pc, sc, cc, rc, configCur, parent_mesh)
configCur.solver.update(
    {
        "final_t": 1,
        "initial_dt": 0.01,
        "time_precision": 6,
        "use_snes": True,
        "print_assembly": False,
    }
)

modelCur.initialize(initialize_solver=False)
modelCur.initialize_discrete_variational_problem_and_solver()
# Write initial condition(s) to file
results = dict()
os.makedirs(f"resultsRect", exist_ok=True)
for species_name, species in modelCur.sc.items:
    results[species_name] = d.XDMFFile(
        modelCur.mpi_comm_world, f"resultsRect/{species_name}.xdmf"
    )
    results[species_name].parameters["flush_output"] = True
    results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)

# Solve
while True:
    # Solve the system
    modelCur.monolithic_solve()
    # Save results for post processing
    for species_name, species in modelCur.sc.items:
        results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)
    # End if we've passed the final time
    if modelCur.t >= modelCur.final_t:
        break

# compute steady state solution at the end of each run
dx = d.Measure("dx",domain = modelCur.cc['PM1'].dolfin_mesh)
int_val = d.assemble(modelCur.sc['A'].u['u']*dx)
SA = d.assemble(1.0*dx)
ss_val = int_val / SA

2023-05-15 14:27:56,631 smart.model_assembly - DEBUG - Creating dolfin object for space-dependent initial condition A (model_assembly.py:675) 
 2023-05-15 14:27:57,175 smart.mesh - INFO - HDF5 mesh, "parent_mesh", successfully loaded from file: rect_mesh/rect_mesh.h5! (mesh.py:205) 



 2023-05-15 14:27:57,203 smart.model - DEBUG - Checking validity of model (step 1 of ZZ) (model.py:209) 


2023-05-15 14:27:57,207 smart.model - DEBUG - Check that mesh/compartment dimensionalities match (model.py:285) 
2023-05-15 14:27:57,213 smart.model - DEBUG - Checking for namespace conflicts (model.py:314) 
 2023-05-15 14:27:57,216 smart.model - DEBUG - Step 1 of initialization completed successfully! (model.py:213) 



 2023-05-15 14:27:57,219 smart.model - DEBUG - Cross-Container Dependent Initializations (step 2 of ZZ) (model.py:221) 


2023-05-15 14:27:57,225 smart.model - DEBUG - Turning reactions into unsigned symbolic flux strings (model.py:368) 
2023-05-15 14:27:57,276 smart.model - DEBUG -

╒════╤═════════════╤══════╤══════════════════╤═════════════╤══════════════╤════════════════╕
│    │ name        │   id │   dimensionality │   num_cells │   num_facets │   num_vertices │
╞════╪═════════════╪══════╪══════════════════╪═════════════╪══════════════╪════════════════╡
│  0 │ parent_mesh │    7 │                2 │        1000 │         1560 │            561 │
├────┼─────────────┼──────┼──────────────────┼─────────────┼──────────────┼────────────────┤
│  1 │ junction    │   21 │                1 │          10 │           11 │             11 │
├────┼─────────────┼──────┼──────────────────┼─────────────┼──────────────┼────────────────┤
│  2 │ PM1         │   27 │                2 │         500 │          785 │            286 │
╘════╧═════════════╧══════╧══════════════════╧═════════════╧══════════════╧════════════════╛


2023-05-15 14:27:58,097 smart.common - DEBUG - snes residual assemble (iter 3) finished in 0.001348 s (common.py:189) 
2023-05-15 14:27:58,103 smart.common - DEBUG - snes residual assemble (iter 4) finished in 0.001287 s (common.py:189) 
 2023-05-15 14:27:58,109 smart.model - DEBUG -   1 SNES Function norm 1.022832e-14 (model.py:1067) 


...........................................................................................................................
..... 2023-05-15 14:27:58,113 smart.model - INFO - Completed time-step 1 [time=0.010000, dt=0.010000] (model.py:1537) .....
...........................................................................................................................


2023-05-15 14:27:58,116 smart.common - DEBUG - snes initialize zero matrices finished in 0 s (common.py:205) 
2023-05-15 14:27:58,119 smart.common - DEBUG - snes jacobian assemble        finished in 0.001278 s (common.py:205) 
2023-05-15 14:27:58,121 smart.common - DEBUG - snes residua

In [ ]:
# %matplotlib inline
# plt.plot(radiusVec, ss_vec, 'ro')
# radiusTest = np.logspace(0,1,100)
# thieleMod = radiusTest / 1.0
# k_kin = 50
# k_p = 10
# cT = 1
# D = 10
# C1 = k_kin*cT*radiusTest**2/((3*D*(np.sqrt(k_p/D)-(1/radiusTest)) + k_kin*radiusTest)*np.exp(thieleMod) +
#         (3*D*(np.sqrt(k_p/D)+(1/radiusTest))-k_kin*radiusTest)*np.exp(-thieleMod))
# cA = (6*C1/radiusTest)*(np.cosh(thieleMod)/thieleMod - np.sinh(thieleMod)/thieleMod**2)
# plt.plot(radiusTest, cA)
# plt.xlabel("Cell radius (μm)")
# plt.ylabel("Steady state concentration (μM)")
# plt.legend(("SMART simulation", "Analytical solution"))